In [3]:
import torch
import torch.nn as nn
import math
import torch.nn.functional as F

In [ ]:
 ### Sample to just understand the multihead

context_length = 5 ## just a sentence length
batch_size = 1
n_embd = 12
n_head = 3
dropout = 0.2

### Two Approaches to Multi-Head Attention

#### 1. Single Linear Layer (Industry Standard Approach)
- In this method, **one Linear layer** generates **Q, K, and V together**.
- The layer projects the input from `d_model → 3 × d_model` in a single forward pass.
- After that, the output is **reshaped and split into multiple heads**.
- This approach is **fast and efficient**, which is why it is used in almost all production-grade Transformer models (e.g., GPT, BERT, etc.).

#### 2. Separate Per-Head Linear Layers (Parallel Head Construction)
- In this method, **each attention head has its own independent Linear layers** for Q, K, and V.
- Heads operate **independently**, and their outputs are **combined at the end**.
- This approach is **conceptually simpler**, but **slower** because:
  - More linear layers
  - More parameter copies
  - Less efficient GPU batching

---

### ✅ Why Approach 1 is Preferred

1. **Efficiency / Speed**
   - One linear layer for QKV means **fewer matrix multiplications** than separate layers.
   - Computation is done in a **single large matrix operation**, reducing overhead and improving GPU parallelism.

2. **Industry Usage**
   - Most production models (e.g., GPT, BERT) use this method because it is **fast and memory-efficient**.
   - The concatenated heads’ weights are implemented with a **single linear layer** for simplicity and scalability.

3. **Memory Efficiency**
   - Fewer linear layers → **fewer parameters** → **less memory usage**, which is critical for large-scale models.

---

### ❗ Caveats / When Separate Heads Might Make Sense

- Separate per-head layers offer **more flexibility**: each head can learn **very different projections**.
- For **small models or teaching purposes**, the performance difference is **minimal**.
- **Specialized architectures** or **memory-constrained inference** may use variants like **multi-query attention**.
- The single-linear approach **assumes all heads have equal dimension**; if you need **different head sizes**, separate layers are required.

---

### 🔎 Summary / Conclusion

- ✅ **Single Linear for QKV** is widely used in industry: **faster**, **memory-efficient**, and **scalable**.
- ⚠️ **Separate per-head layers** are **slower** and mostly used for **teaching** or **research experiments**.
- While **Approach 1 dominates in practice**, **Approach 2** can be useful for **custom or experimental models**.

## First approach to build the Multi Head 

In [8]:
class MultiheadAttention(nn.Module):
    def __init__(self, n_head: int, d_model: int, context_length: int, dropout: float = 0.0, log_shape: bool = True):
        super().__init__()

        self.log_shape = log_shape
        self.n_head = n_head
        self.d_model = d_model
        
        # Each attention head must get the same number of dimensions.
        # Example: d_model = 12 and n_head = 3 → d_head = 4
        assert d_model % n_head == 0, "Embedding size must be divisible by number of heads"
        self.d_head = d_model // n_head

        # This linear layer produces Q, K, and V together.
        # Input:  (B, T, d_model)
        # Output: (B, T, 3*d_model)
        # Because we need Q, K, V — each of size d_model.
        self.qkv = nn.Linear(d_model, 3 * d_model, bias=False)
        
        # After attention finishes, all heads are merged back
        # and passed through this output projection.
        self.projection = nn.Linear(d_model, d_model, bias=False)
        self.dropout = nn.Dropout(dropout)

        # Causal mask:
        # Upper triangular matrix with True above diagonal.
        # This prevents a token from looking at future tokens.
        self_mask = torch.triu(
            torch.ones(context_length, context_length, dtype=torch.bool),
            diagonal=1
        )
        # register_buffer ensures this mask moves with the model (GPU/CPU)
        self.register_buffer("causal_mask", self_mask)
        
        
    def forward(self, x: torch.Tensor):
        B, T, D = x.shape

        '''
        Here we have 3 self-attention heads.  
        Each head needs its own Q, K, V vectors, and all of them must have the same size.

        1. We pass 1 sentence → it has 5 tokens → each token has a 12-dimensional embedding.
        So the input shape is: (1, 5, 12)

        2. The qkv() layer is a Linear layer that creates all three matrices:
            Q, K, and V
        It does this by projecting the input from:
            d_model → 3 * d_model
        So output shape becomes: (1, 5, 36)

        3. Now the 36 channels can be evenly divided into 3 heads.
        Because:
            d_model = 12
            n_heads = 3
            head_dim = 12 / 3 = 4

        4. Therefore, each head will receive:
            Q of size 4,
            K of size 4,
            V of size 4
        and this is repeated for all 3 heads.

        In short:
        - Input:       (1, 5, 12)
        - After qkv(): (1, 5, 36)
        - Split into heads: 3 heads × 4 dims each for Q, K, V
        '''
        
        
        qkv = self.qkv(x)
        qkv = qkv.view(B, T, 3, self.n_head, self.d_head)

        if self.log_shape:
            print(f"Shape of QKV: {qkv.shape}")

        # Step 3: Split into Q, K, V
        # Each has shape (B, T, n_head, d_head)
        q, k, v = qkv.unbind(dim=2)

        '''
    bcz  at present -> (B, T, n_head, d_head). and the multihead attention wants the (B, n_head, T, d_head)
    
    q = (B, heads, T, d_head)
    k = (B, heads, T, d_head)
    v = (B, heads, T, d_head)
    
    Each attention head now has access to all T tokens, each represented as a d_head-dim vector.
    
    '''
        q = q.transpose(1, 2)
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)

        if self.log_shape:
            print("q:", q.shape, "k:", k.shape, "v:", v.shape)

        # scale factor = 1 / sqrt(d_head)
        scale = 1.0 / math.sqrt(self.d_head)
        
        
        '''
        k.transpose(-2, -1):  (B, heads, d_head, T)
    
        (B,heads,T,d_head) 
            @
        (B,heads,d_head,T)
        --------------------------------
        → (B, heads, T, T)
        
        attention score matrix of T*T :->  each head computes a 5*5 matrix 
            scores[i,j] = how much token i attends to token j
            
        scake = 1 / sqrt(d_head)   ->  This avoids very large dot-products which would make softmax explode
        
        '''
        attention_weight = torch.matmul(q, k.transpose(-2, -1)) * scale

        
        #  token positions by setting them to -inf
        attention_weight = attention_weight.masked_fill(
            self.causal_mask[:T, :T], 
            float('-inf')
        )

        #  Convert scores to probabilities
        # Softmax applied on last dim so each row sums to 1
        attention_weight_prob = F.softmax(attention_weight, dim=-1)
        attention_weight_prob = self.dropout(attention_weight_prob)

        #Multiply probabilities with V to compute context
        # result shape = (B, n_head, T, d_head)
        context = torch.matmul(attention_weight_prob, v)

        if self.log_shape:
            print("weights:", attention_weight_prob.shape, "context:", context.shape)

        '''
        ctx: (B, heads, T, d_head)
        We need to combine them into:
        (B, T, d_model) = (1, 5, 12)
        
        
        1. transpose back -> (B, T, heads, d_head)
        2. reshape to (B, T, d_model)
        '''
        out = context.transpose(1, 2).contiguous().view(B, T, self.d_model)

        # linear projection
        out = self.projection(out)

        if self.log_shape:
            print("Multi-Head output:", out.shape)

        return out, attention_weight_prob

## Second way to build the Multi Head

In [ ]:
class Head(nn.Module):
    """ One head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)

        # causal mask: lower triangular matrix
        self.register_buffer('tril', torch.tril(torch.ones(context_length, context_length)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # x shape: (B, T, D) -> batch, token length, Dimension of one token
        B, T, D = x.shape

        # project inputs to key, query, value
        k = self.key(x)    # (B, T, hs)
        q = self.query(x)  # (B, T, hs)

        # compute attention scores ("affinities")
        attention_weight = q @ k.transpose(-2, -1) * (k.shape[-1] ** -0.5)   # (B, T, T)

        # apply causal mask to prevent looking ahead
        attention_weight = attention_weight.masked_fill(self.tril[:T, :T] == 0, float('-inf'))

        # softmax to get attention probabilities
        attention_weight_prob = F.softmax(attention_weight, dim=-1)   # (B, T, T)
        attention_weight_prob = self.dropout(attention_weight_prob)

        # weighted sum of values
        v = self.value(x)                               # (B, T, hs)
        out = attention_weight_prob @ v                  # (B, T, hs)

        return out
    
    
    
#      nn.Sequential: layers are applied in order, automatically in forward.
#      nn.ModuleList: just stores the layers, but you decide manually in forward how to use them (more flexible).

class MultiHeadAttention(nn.Module):
    ''' Multiple heads of self-Attention in parallel '''

    def __init__(self, num_head, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_head)])
        self.proj = nn.Linear(num_head*head_size , n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [10]:
vocab = 13
d_model = 4

In [ ]:
'''
	•	Token embeddings: Each token in the vocabulary has a unique learnable vector. Shape = (vocab_size, d_model).
	•	Example: "apple" and "orange" get completely different learned embeddings.
	•	Positional embeddings: Each position in the sequence (0, 1, 2, …, context_length-1) has a unique vector. Shape = (context_length, d_model).
	•	Example: position 0 always has the same embedding vector, regardless of which token is there.
	•	It is not per-token, its per-position.
 
 only the context length positions are trained for positional embeddings, not every token individually
'''


class PositionalEmbeddings(nn.Module):
    def __init__(self, vocab: int , d_model: int, device):
        super().__init__()
        self.pos_embeddings = nn.Embedding(context_length,d_model, device=device)
        
    def forward(self, x:torch.Tensor):
        pos_emb = self.pos_embeddings(x)
        return pos_emb

In [31]:
pos = PositionalEmbeddings(vocab,d_model,device="cpu")

## Tokenizer

In [ ]:
## simple tokenizer

class byteTokenizer:
    def __init__(self, file_path: str):
        with open(file_path, 'r', encoding='utf-8') as f:
            self.text = f.read()
        
        # Create sorted list of unique characters
        self.chars = sorted(set(self.text))
        print("Unique characters:", self.chars)
        print(f"Length of unique chars: {len(self.chars)}")
        
        # Mapping: char -> int and int -> char
        self.char_to_int = {ch: i for i, ch in enumerate(self.chars)}
        self.int_to_char = {i: ch for i, ch in enumerate(self.chars)}


    @property
    def vocab_size(self)-> int:
        return len(self.chars)
        
        
    def encode(self, s: str) -> list[int]:
        """Convert string to list of integers"""
        return [self.char_to_int[c] for c in s]

    def decode(self, l: list[int]) -> str:
        """Convert list of integers back to string"""
        return ''.join([self.int_to_char[i] for i in l])


tokenizer = byteTokenizer("/Users/abhaykumarsingh/Desktop/Gpt/Data/tulsidas.txt")
encoded = tokenizer.encode("नमस्ते")  
decoded = tokenizer.decode(encoded)

print("Encoded:", encoded)
print("Decoded:", decoded)

Unique characters: ['\n', ' ', '(', ')', ',', '-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'í', 'ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ए', 'ऐ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह', '़', 'ऽ', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'े', 'ै', 'ो', 'ौ', '्', '।', '॥', '०', '–']
Length of unique chars: 80
Encoded: [49, 54, 61, 75, 45, 71]
Decoded: नमस्ते


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Optional


class Decoder_Multi_Head_Attention(nn.Module):
    def __init__(self, n_head: int, d_model:int, dropout:float=0.0):
        super().__init__()
        assert d_model % n_head == 0   # should be divisible (d_model % n_head = 0)
        self.n_head = n_head
        self.d_head = d_model // n_head
        self.qkv = nn.Linear(d_model, 3 * d_model, bias=False)
        self.proj = nn.Linear(d_model, d_model, bias=False)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x:torch.Tensor) -> None:
        B, T, C = x.shape
        qkv = self.qkv(x).view(B, T, 3, self.n_head, self.d_head)
        q, k, v = qkv.unbind(dim=2)
        q = q.transpose(1, 2)
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)
        out = F.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout.p if self.training else 0.0, is_causal=True)
        out = out.transpose(1, 2).contiguous().view(B, T, C)
        out = self.proj(out)
        return out
    
## End Feed Forward of One Block 
class FeedForward(nn.Module):
    def __init__(self, d_model: int, hidden_times: int = 3, dropout: float=0.0) -> None:
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_model, hidden_times*d_model),
            nn.GELU(),
            nn.Linear(hidden_times*d_model, d_model),
            nn.Dropout(dropout)
        )
    
    def forward(self, x: torch.Tensor):
        return self.net(x)
        
## one single tranformer block       
class Block(nn.Module):
    def __init__(self, d_model: int, n_head: int, dropout: float=0.2) -> None:
        super().__init__()
        self.lr1Norm = nn.LayerNorm(d_model)
        self.multiHead_attention = Decoder_Multi_Head_Attention(n_head, d_model, dropout)
        self.lr2Norm = nn.LayerNorm(d_model)
        self.ffn = FeedForward(d_model, dropout=dropout)
        
    def forward(self, x: torch.Tensor):
        x = x + self.multiHead_attention(self.lr1Norm(x))
        x = x + self.ffn(self.lr2Norm(x))
        
        return x
    
    

## multi block architecture - GPT

class TinyGPT(nn.Module):
    def __init__(self, vocab_size: int, context_length: int, n_block: int = 4, n_head: int = 4, d_model: int = 256, dropout: float = 0.0):
        super().__init__()
        self.embedings = nn.Embedding(vocab_size, d_model)
        self.d_model = d_model
        self.context_length = context_length
        self.pos_embedings = nn.Embedding(context_length, d_model)
        self.blocks = nn.Sequential(*[Block(d_model, n_head, dropout) for _ in range(n_block)])
        self.out_head = nn.Linear(d_model, vocab_size)
        self.lrNorm = nn.LayerNorm(d_model)
        
        self.apply(self._init_weights)
        
    # initialising the weights in normal distribution near zero - this results better than random initialising 
    
    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.normal_(m.weight, mean=0.0, std= 0.02)
            if m.bias is not None:
                nn.init.zeros_(m.bias)
        elif isinstance(m, nn.Embedding):
            nn.init.normal_(m.weight, mean=0.0, std=0.02)
            
    
    
        
    def forward(self, idx: torch.Tensor, targets: Optional[torch.Tensor]=None):
        B, T = idx.shape
        if T > self.context_length:
            idx = idx[:, -self.context_length:]
            T = idx.size(1)
        
        pos = torch.arange(0, T, device=idx.device).unsqueeze(0)
        x = self.embedings(idx) + self.pos_embedings(pos)
        x = self.blocks(x)
        x = self.lrNorm(x)
        logits = self.out_head(x)
        loss = None
        if targets is not None :
            loss = F.cross_entropy(logits.flatten(0, 1), targets.flatten())
            
        return logits, loss